In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Corporate Actions in LUSID

Attributes
----------
Chart of Accounts
Accounts
Properties
"""

toggle_code("Toggle Docstring")

# Create and Setup Chart of Accounts and General Ledger Accounts

This notebook demonstrates how to create multiple Charts of Accounts and multiple General Ledger Accounts to populate them with from data in an excel file.

#### In this notebook:
- We create example [properties](https://support.lusid.com/knowledgebase/article/KA-01855/en-us) for both the ChartOfAccounts and Account domains.  
- We create 3 Charts of Accounts from an example data sheet.
- We update the properties of these Charts of Accounts from another example data sheet.
- We create multiple General Ledger accounts from an example data sheet and link to the 3 Charts of Accounts created.
- We update the properties of a selection of these General Ledger Accounts from an example data sheet.

#### Required Licences: 
- lusid-abor-all-status-access
- lusid-website-feature-fundaccounting

### Table of contents
* [Setup](#setup)
* [1. Load Data](#load-data)
* [2. Create Properties](#create-properties)
* [3. Chart of Accounts](#chart-of-accounts)
* [4. Accounts](#accounts)
* [5. Read From LUSID](#read-from-lusid)

## Setup<a name = "setup"></a>

In [2]:
# Import lusid specific packages
# These are the core lusid packages for interacting with the API via Python
import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory

from lusidjam.refreshing_token import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame

import pandas as pd
import json
import os

pd.set_option("display.max_columns", None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
lusid_api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

In [3]:
# Define the apis
coa_api = lusid_api_factory.build(lusid.ChartOfAccountsApi)
property_definitions_api = lusid_api_factory.build(lusid.PropertyDefinitionsApi)

## 1. Load Data <a name = "load-data"></a>

Load data from an excel workbook from the following 4 tabs:

- Chart Of Accounts
- Chart Of Accounts Properties
- Accounts
- Accounts Properties

In [4]:
# Chart of Accounts DF column data types
coa_df_columns = {
            "Scope": str,
            "Code": str,
            "Name": str,
            "Description": str,
            "LocalCountryReference": str
    }

# Chart of Accounts Properties DF column data types
coa_p_df_columns = {
            "Scope": str,
            "Code": str,
            "Key": str,
            "Value": str
    }


# Accounts DF column data types
a_df_columns = {
            "COA Scope": str,
            "COA Code": str,
            "Code": str,
            "Description": str,
            "Type": str,
            "Status": str,
            "Control": str,
            "DescriptionOverrides": str
    }

# Accounts Properties DF column data types
a_p_df_columns = {
            "COA Scope": str,
            "COA Code": str,
            "Code": str,
            "Key": str,
            "Value": str
    }


# Read data from file
file_name = "chart-of-accounts-data.xlsx"
file_location = f"{os.getcwd()}/data/{file_name}"

chart_of_accounts_df = pd.read_excel(file_location, "Chart Of Accounts", dtype=coa_df_columns).dropna(how='all')
chart_of_accounts_df.fillna("", inplace = True)
chart_of_accounts_props_df = pd.read_excel(file_location, "Chart Of Accounts Properties", dtype=coa_p_df_columns).dropna(how='all')
chart_of_accounts_props_df.fillna("", inplace = True)
accounts_df = pd.read_excel(file_location, "Accounts", dtype=a_df_columns).dropna(how='all')
accounts_df.fillna("", inplace = True)
accounts_props_df = pd.read_excel(file_location, "Accounts Properties", dtype=a_p_df_columns).dropna(how='all')
accounts_props_df.fillna("", inplace = True)

## 2. Create Properties <a name = "create-properties"></a>

In [5]:
# Chart of Accounts Properties
try:
    response = property_definitions_api.create_property_definition(
        models.CreatePropertyDefinitionRequest(domain="ChartOfAccounts",
                                               scope="coa-testing",
                                               code="LocalCountryReference",
                                               display_name="LocalCountryReference",
                                               data_type_id=lusid.ResourceId(scope="system",code="string"),
                                               life_time="TimeVariant",
                                               constraint_style="Property",
                                               property_description="LocalCountryReference"))
    print(response)
    
except lusid.ApiException as e:
        print(json.loads(e.body)["title"])
        

# Account Properties
try:
    response = property_definitions_api.create_property_definition(
        models.CreatePropertyDefinitionRequest(domain="Account",
                                               scope="coa-testing",
                                               code="DescriptionOverrides",
                                               display_name="DescriptionOverrides",
                                               data_type_id=lusid.ResourceId(scope="system",code="string"),
                                               life_time="TimeVariant",
                                               constraint_style="Property",
                                               property_description="DescriptionOverrides"))
    print(response)
    
except lusid.ApiException as e:
        print(json.loads(e.body)["title"])

Error creating Property Definition 'ChartOfAccounts/coa-testing/LocalCountryReference' because it already exists.
Error creating Property Definition 'Account/coa-testing/DescriptionOverrides' because it already exists.


## 3. Chart of Accounts <a name = "chart-of-accounts"></a>

A Chart of Accounts lists all the accounts that can be used to record double entry accounting in a general ledger. It is broken down into subcategories - assets, liabilities, equity, expenses and revenue.

### 3.1 [Create Charts of Accounts](https://www.lusid.com/docs/api/#operation/CreateChartOfAccounts)

In [6]:
# Uncomment to delete COAs

# coa_api.delete_chart_of_accounts(scope="coa-testing", code="coa-1")
# coa_api.delete_chart_of_accounts(scope="coa-testing", code="coa-2")
# coa_api.delete_chart_of_accounts(scope="coa-testing", code="coa-3")

In [7]:
# Iterate through Charts of Accounts data and try to create them
for i in range(len(chart_of_accounts_df)):    
    try: 
        response = coa_api.create_chart_of_accounts(scope=chart_of_accounts_df['Scope'][i],
                                                chart_of_accounts_request=models.ChartOfAccountsRequest(code=chart_of_accounts_df['Code'][i],
                                                                         name=chart_of_accounts_df['Name'][i],
                                                                         description=chart_of_accounts_df['Description'][i],
                                                                         properties={"chartOfAccounts/coa-testing/LocalCountryReference": {"key":"chartOfAccounts/coa-testing/LocalCountryReference",
                                                                                 "value":
                                                                                 {"labelValue":chart_of_accounts_df['LocalCountryReference'][i]}}}))
#         print(response)
    
    except lusid.ApiException as e:
        print(json.loads(e.body)["title"])


### 3.2 [Upsert (Update) Chart Of Accounts Properties](https://www.lusid.com/docs/api/#operation/UpsertChartOfAccountsProperties)

In [8]:
# Define unique Chart of Accounts identifier column made up of COA scope & COA code
chart_of_accounts_props_df['Scope:Code'] = chart_of_accounts_props_df['Scope'] + ':' + chart_of_accounts_props_df['Code']

# Isolate unique Charts of Accounts from new column
unique_coas = set(chart_of_accounts_props_df['Scope:Code'].tolist())


# Iterate through unique Charts of Accounts and update properties
for coa in unique_coas:
    chart_of_accounts_props_df_single = chart_of_accounts_props_df.loc[chart_of_accounts_props_df['Scope:Code'] == coa].reset_index()
    chart_of_accounts_props = {}
    for i in range(len(chart_of_accounts_props_df_single)):
        
        chart_of_accounts_props[f"{chart_of_accounts_props_df_single['Key'][i]}"] = {"key":chart_of_accounts_props_df_single['Key'][i],
                                                                                     "value":
                                                                                     {"labelValue":chart_of_accounts_props_df_single['Value'][i]}}
        
    
    response = coa_api.upsert_chart_of_accounts_properties(scope=coa.split(":")[0],
                                                       code=coa.split(":")[1],
                                                       request_body=chart_of_accounts_props)
#     print(response)

## 4. Accounts <a name = "accounts"></a>

An account is a separate record within the general ledger that is assigned to specific asset, liability, equity, revenue or expense items. It is used to record transactions over a period of time.

### 4.1 [Upsert Accounts](https://www.lusid.com/docs/api/#operation/UpsertAccounts)

Upsert General Ledger accounts 

In [9]:
# Define unique Chart of Accounts identifier column made up of COA scope & COA code
accounts_df['COA Scope:Code'] = accounts_df['COA Scope'] + ':' + accounts_df['COA Code']

# Isolate unique Charts of Accounts from new column
unique_coas = set(accounts_df['COA Scope:Code'].tolist())


# Iterate through unique Charts of Accounts and upsert accounts
for coa in unique_coas:
    accounts_df_single = accounts_df.loc[accounts_df['COA Scope:Code'] == coa].reset_index()
    accounts = []
    for i in range(len(accounts_df_single)):
        accounts.append(models.Account(code=accounts_df_single['Code'][i],
                                       description=accounts_df_single['Description'][i],
                                       type=accounts_df_single['Type'][i],
                                       status=accounts_df_single['Status'][i],
                                       control=accounts_df_single['Control'][i],
                                       properties={"account/coa-testing/DescriptionOverrides": {"key":"account/coa-testing/DescriptionOverrides",
                                                                                 "value": {"labelValue":accounts_df['DescriptionOverrides'][i]}}}))
    response = coa_api.upsert_accounts(scope=coa.split(":")[0],
                                       code=coa.split(":")[1],
                                       account=accounts)
#     print(response)

### 4.2 [Upsert (Update) Account Properties](https://www.lusid.com/docs/api/#operation/UpsertAccountsProperties)

In [10]:
# Define unique Account identifier column made up of COA scope, COA code & A code
accounts_props_df['COAScope:COACode:ACode'] = accounts_props_df['COA Scope'] + ':' + accounts_props_df['COA Code'] + ':' + accounts_props_df['Code']

# Isolate unique Accounts from new column
unique_as = set(accounts_props_df['COAScope:COACode:ACode'].tolist())


# Iterate through unique Accounts and properties
for a in unique_as:
    accounts_props = {}
    accounts_df_props_single = accounts_props_df.loc[accounts_props_df['COAScope:COACode:ACode'] == a].reset_index()
    for i in range(len(accounts_df_props_single)):
        accounts_props[f"{accounts_df_props_single['Key'][i]}"] = {"key":accounts_df_props_single['Key'][i],
                                                                                     "value":
                                                                                     {"labelValue":accounts_df_props_single['Value'][i]}}
    response = coa_api.upsert_accounts_properties(scope=a.split(":")[0],
                                                  code=a.split(":")[1],
                                                  account_code=a.split(":")[2],
                                                  request_body=accounts_props)
    
#     print(response)
        

## 5. Read From LUSID <a name = "read-from-lusid"></a>

Once the data entities are written into LUSID, they can be access via the UI or via API directly as below.

### 5.1 List Charts Of Accounts

In [11]:
# List all Charts of Accounts
coa_list_df = lusid_response_to_data_frame(
    coa_api.list_charts_of_accounts(property_keys=["chartOfAccounts/coa-testing/LocalCountryReference"])).drop(['href', 'links.0.href'], axis=1)

coa_list_df

,id.scope,id.code,name,description,properties.ChartOfAccounts/coa-testing/LocalCountryReference.key,properties.ChartOfAccounts/coa-testing/LocalCountryReference.value.label_value,properties.ChartOfAccounts/coa-testing/LocalCountryReference.effective_from,properties.ChartOfAccounts/coa-testing/LocalCountryReference.effective_until,version.effective_from,version.as_at_date,links.0.relation,links.0.method
0,coa-testing,coa-1,coa-1,coa-1,ChartOfAccounts/coa-testing/LocalCountryReference,APAC,0001-01-01 00:00:00+00:00,9999-12-31 23:59:59.999999+00:00,0001-01-01 00:00:00+00:00,2023-01-27 10:37:00.460115+00:00,EntitySchema,GET
1,coa-testing,coa-2,coa-2,coa-2,ChartOfAccounts/coa-testing/LocalCountryReference,APAC,0001-01-01 00:00:00+00:00,9999-12-31 23:59:59.999999+00:00,0001-01-01 00:00:00+00:00,2023-01-27 10:37:00.879101+00:00,EntitySchema,GET
2,coa-testing,coa-3,coa-3,coa-3,ChartOfAccounts/coa-testing/LocalCountryReference,APAC,0001-01-01 00:00:00+00:00,9999-12-31 23:59:59.999999+00:00,0001-01-01 00:00:00+00:00,2023-01-27 10:37:01.119424+00:00,EntitySchema,GET


### 5.2 List Accounts

In [12]:
# List all Accounts for a specific Chart of Accounts scope and code
a_list_df = lusid_response_to_data_frame(
    coa_api.list_accounts(scope="coa-testing", 
                          code="coa-1",
                          property_keys=["account/coa-testing/DescriptionOverrides"]))

a_list_df[:10]

,code,description,type,status,control,properties.Account/coa-testing/DescriptionOverrides.key,properties.Account/coa-testing/DescriptionOverrides.value.label_value,properties.Account/coa-testing/DescriptionOverrides.effective_from
0,100100,Long Investment Cost - Common Stock,Asset,Active,System,Account/coa-testing/DescriptionOverrides,,0001-01-01 00:00:00+00:00
1,109900,Long Investment Cost - Other Investment,Asset,Active,Manual,Account/coa-testing/DescriptionOverrides,,0001-01-01 00:00:00+00:00
2,110100,Long Unrealized Price Gain/Loss - Common Stock,Asset,Active,System,Account/coa-testing/DescriptionOverrides,,0001-01-01 00:00:00+00:00
3,112100,Long Unrealized FX Gain/Loss - Common Stock,Asset,Active,System,Account/coa-testing/DescriptionOverrides,,0001-01-01 00:00:00+00:00
4,119000,Unrealized gain/loss - Other Investment,Asset,Active,Manual,Account/coa-testing/DescriptionOverrides,,0001-01-01 00:00:00+00:00
5,120000,Citibank London - Cash Account,Asset,Active,Manual,Account/coa-testing/DescriptionOverrides,,0001-01-01 00:00:00+00:00
6,141000,Dividend Income Receivable,Asset,Active,Manual,Account/coa-testing/DescriptionOverrides,,0001-01-01 00:00:00+00:00
7,142200,Interest Receivable - Fixed Rate Bond,Asset,Active,System,Account/coa-testing/DescriptionOverrides,,0001-01-01 00:00:00+00:00
8,143000,Bank Interest Receivable,Asset,Active,Manual,Account/coa-testing/DescriptionOverrides,,0001-01-01 00:00:00+00:00
9,145000,Time Desposit Interest Receivable,Asset,Active,Manual,Account/coa-testing/DescriptionOverrides,,0001-01-01 00:00:00+00:00


## 6. Clean Up

In [13]:
# Delete Charts of Accounts

try:
    coa_api.delete_chart_of_accounts(scope="coa-testing", code="coa-1")
    coa_api.delete_chart_of_accounts(scope="coa-testing", code="coa-2")
    coa_api.delete_chart_of_accounts(scope="coa-testing", code="coa-3")
    print("Charts of Accounts deleted")
except lusid.ApiException as e:
        print(json.loads(e.body)["title"])

Charts of Accounts deleted
